## Meta Data Analysis file

This file contains the updated analysis of the meta data that enables the variables of interest to be pulled off in the relevant models.

For modelling purposes need the data to:

1. indicate null values - suppressed, lowcoverage or ' ' for missing with a Nan in pandas dataframe;
2. discard the 95% conidence interval columns;
3. discard columsn that have null values that I cannot imput as I have no knowledge for this;

In [ ]:
#Only run the following if you have a new folder and no access to the input files
#read in csv meta data files from githun!curl https://raw.githubusercontent.com/Trakky1/Notes_DAT17/master/census_meta.csv -o census_meta.csv
!curl https://raw.githubusercontent.com/Trakky1/Notes_DAT17/master/census_meta.csv -o census_meta.csv
!curl https://raw.githubusercontent.com/Trakky1/Notes_DAT17/master/ks4_meta.csv -o ks4_meta.csv

In [1]:
#import libraries
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
#printing of multiple outputs from a cell
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#put in dataframe
census_meta=pd.read_csv('census_meta.csv', na_values=['NP',' ','SUPP','NE','LOWCOV',None])
ks4_meta=pd.read_csv('ks4_meta.csv', na_values=['NP',' ','SUPP','NE','LOWCOV',None], low_memory=False)

Below is the meta data for the school census files

In [3]:
census_meta

,Variable,Label,Type,Description
0,URN,Unique Reference Number,NaN,NaN
1,LA,LA Number,NaN,NaN
2,ESTAB,Establishment number,NaN,NaN
3,NOR,Total number of pupils on roll,INTEGER,NaN
4,NORG,Number of girls on roll,INTEGER,NaN
5,NORB,Number of boys on roll,INTEGER,NaN
6,PNORG,Percentage of girls on roll,PCT1,NaN
7,PNORB,Percentage of boys on roll,PCT1,NaN
8,TSENELSE,Number of SEN pupils with a statement or EHC plan,INTEGER,NaN
9,PSENELSE,Percentage of SEN pupils with a statement or E...,PCT1,NaN


In [4]:
ks4_meta.count()
ks4_meta.head()

Column                                                  372
Metafile heading                                        372
Metafile description                                    372
Methodology changes                                      25
Null field for special schools                            4
Null field for local authority records                   79
Null field for National (all schools) records           284
Null field for National (maintained schools) records     77
Unnamed: 8                                                0
Unnamed: 9                                                0
dtype: int64

,Column,Metafile heading,Metafile description,Methodology changes,Null field for special schools,Null field for local authority records,Null field for National (all schools) records,Null field for National (maintained schools) records,Unnamed: 8,Unnamed: 9
0,1,RECTYPE,Record type (1=mainstream school; 2=special sc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ALPHAIND,Alphabetic sorting index,NaN,NaN,Yes,Yes,Yes,NaN,NaN
2,3,LEA,Local authority code (see separate list of loc...,NaN,NaN,NaN,Yes,Yes,NaN,NaN
3,4,ESTAB,Establishment number,NaN,NaN,Yes,Yes,Yes,NaN,NaN
4,5,URN,School Unique Reference Number,NaN,NaN,Yes,Yes,Yes,NaN,NaN


The Metafile heading column contains the list of all features of interest. I will drop the confidence interval columns and the columns with text as well-_they cannot go through a random forest model_.

In [5]:
#helper function to return index values
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]

In [8]:
#find the indices for the columsn of interest
pd.set_option('display.max_colwidth', -1)
CI_int=ks4_meta.index[ks4_meta['Metafile description'].str.contains('confidence interval')]
CI_meta=ks4_meta.loc[CI_int]
CI_meta[['Metafile heading','Metafile description']]

,Metafile heading,Metafile description
72,P8CILOW,Lower 95% confidence interval for Progress 8 measure
73,P8CIUPP,Upper 95% confidence interval for Progress 8 measure
75,P8MEAENG_CILOW,Lower 95% confidence interval for Progress 8 English element
76,P8MEAENG_CIUPP,Upper 95% confidence interval for Progress 8 English element
78,P8MEAMAT_CILOW,Lower 95% confidence interval for Progress 8 maths element
79,P8MEAMAT_CIUPP,Upper 95% confidence interval for Progress 8 maths element
81,P8MEAEBAC_CILOW,Lower 95% confidence interval for Progress 8 EBacc element
82,P8MEAEBAC_CIUPP,Upper 95% confidence interval for Progress 8 EBacc element
84,P8MEAOPEN_CILOW,Lower 95% confidence interval for Progress 8 open element
85,P8MEAOPEN_CIUPP,Upper 95% confidence interval for Progress 8 open element


In [9]:
#now wrap that up in a list
CI_meta.shape

(46, 10)

In [11]:
perc_lst=CI_meta[['Column','Metafile heading']]

In [18]:
#see disadvantaged pupils metadata
disadv1=ks4_meta.index[(ks4_meta['Metafile description'].str.contains('Progress 8')) & (ks4_meta['Metafile description'].str.contains('disadvantaged pupils')) & ~(ks4_meta['Metafile description'].str.contains('confidence interval'))]
disadv1_desc=ks4_meta.loc[disadv1]
disadv1_desc[['Metafile heading','Metafile description']]

,Metafile heading,Metafile description
136,P8PUP_FSM6CLA1A,Number of disadvantaged pupils in Progress 8 measure
137,P8MEA_FSM6CLA1A,Progress 8 measure - disadvantaged pupils
141,P8PUP_NFSM6CLA1A,Number of non-disadvantaged pupils in Progress 8 measure
142,P8MEA_NFSM6CLA1A,Progress 8 measure - non-disadvantaged pupils
146,P8MEAENG_FSM6CLA1A,Progress 8 measure for English element - disadvantaged pupils
150,P8MEAMAT_FSM6CLA1A,Progress 8 measure for maths element - disadvantaged pupils
154,P8MEAEBAC_FSM6CLA1A,Progress 8 measure for EBacc element - disadvantaged pupils
158,P8MEAOPEN_FSM6CLA1A,Progress 8 measure for open element - disadvantaged pupils
162,P8MEAENG_NFSM6CLA1A,Progress 8 measure for English element - non-disadvantaged pupils
166,P8MEAMAT_NFSM6CLA1A,Progress 8 measure for maths element - non-disadvantaged pupils


In [17]:
#now find the text columns
txtcols=ks4_meta.index[(ks4_meta['Metafile heading'].str.contains('SCHNAME')) | (ks4_meta['Metafile heading'].str.contains('ADDRESS')) | (ks4_meta['Metafile description'].str.contains('phone'))]
txtcols_desc=ks4_meta.loc[txtcols]
txtcols_desc[['Metafile heading','Metafile description']]

,Metafile heading,Metafile description
5,SCHNAME,School name
6,SCHNAME_AC,School now known as (used if the school has converted to an academy on or after 12 Sept 2015)
7,ADDRESS1,School address (1)
8,ADDRESS2,School address (2)
9,ADDRESS3,School address (3)
12,TELNUM,School telephone number


In [24]:
disadv_lst=disadv1_desc[['Column','Metafile heading']]
txtcols_lst=txtcols_desc[['Column','Metafile heading']]
cols_to_drop=pd.concat([perc_lst,txtcols_lst], ignore_index=True)
cols_to_drop.shape

(52, 2)

In [25]:
cols_to_drop.to_csv('cols_to_drop.csv')

In [28]:
#check the parameters returned from the random forest
#df.loc[df['A'] == 'foo']
ks4_meta.loc[ks4_meta['Metafile heading']=='ATT8SCROPEN']

,Column,Metafile heading,Metafile description,Methodology changes,Null field for special schools,Null field for local authority records,Null field for National (all schools) records,Null field for National (maintained schools) records,Unnamed: 8,Unnamed: 9
64,65,ATT8SCROPEN,Average Attainment 8 score per pupil for open element,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#check the parameters returned from the random forest
#df.loc[df['A'] == 'foo']
ks4_meta.loc[ks4_meta['Metafile heading']=='PTL2BASICS_LL_PTQ_EE']

,Column,Metafile heading,Metafile description,Methodology changes,Null field for special schools,Null field for local authority records,Null field for National (all schools) records,Null field for National (maintained schools) records,Unnamed: 8,Unnamed: 9
86,87,PTL2BASICS_LL_PTQ_EE,Percentage of pupils achieving grades A*-C in both English and mathematics GCSEs,Yes - see 2016 methodology sheet for basics methodology change,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#PTL2BASICS_3YR_PTQ_EE	
ks4_meta.loc[ks4_meta['Metafile heading']=='PTL2BASICS_3YR_PTQ_EE']

,Column,Metafile heading,Metafile description,Methodology changes,Null field for special schools,Null field for local authority records,Null field for National (all schools) records,Null field for National (maintained schools) records,Unnamed: 8,Unnamed: 9
87,88,PTL2BASICS_3YR_PTQ_EE,"Three year percentage of pupils achieving A*-C in both English and mathematics GCSEs over 2014, 2015 and 2016",Yes - see 2016 methodology sheet for basics methodology change,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
!curl https://raw.githubusercontent.com/Trakky1/Notes_DAT17/master/ks4_meta_subset.csv -o ks4_meta_subset.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3304  100  3304    0     0   6051      0 --:--:-- --:--:-- --:--:--  6051


In [32]:
ks4_meta_subset=pd.read_csv('ks4_meta_subset.csv', na_values=['NP',' ','SUPP','NE','LOWCOV',None])

In [33]:
ks4_meta_subset

,Column,Metafile heading,Metafile description
0,1,RECTYPE,Record type (1=mainstream school; 2=special school; 4=local authority; 5=National (all schools); 7=National (maintained schools))
1,2,ALPHAIND,Alphabetic sorting index
2,3,LEA,Local authority code (see separate list of local authorities and their codes)
3,4,ESTAB,Establishment number
4,5,URN,School Unique Reference Number
5,16,NFTYPE,School type (see separate list of abbreviations used in the tables)
6,17,RELDENOM,School religious character
7,18,ADMPOL,School admissions policy (self-declared by schools on Edubase)
8,19,EGENDER,School gender of entry
9,29,BPUP,Number of boys at the end of key stage 4
